<a href="https://colab.research.google.com/github/chengren/bky_hw_lab/blob/edu/PyBKT_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [311]:
!apt-get install libboost1.62-dev libboost-mpi-python1.62-dev libboost1.62-tools-dev libboost1.62-all-dev libboost-mpi-python1.62.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-mpi-python1.62-dev is already the newest version (1.62.0+dfsg-5).
libboost-mpi-python1.62.0 is already the newest version (1.62.0+dfsg-5).
libboost1.62-all-dev is already the newest version (1.62.0+dfsg-5).
libboost1.62-dev is already the newest version (1.62.0+dfsg-5).
libboost1.62-tools-dev is already the newest version (1.62.0+dfsg-5).
The following packages were automatically installed and are no longer required:
  libboost-atomic1.65.1 libboost-chrono1.65.1 libboost-container1.65.1
  libboost-context1.65.1 libboost-coroutine1.65.1 libboost-date-time1.65.1
  libboost-fiber1.65.1 libboost-filesystem1.65.1 libboost-graph-parallel1.65.1
  libboost-graph1.65.1 libboost-iostreams1.65.1 libboost-locale1.65.1
  libboost-log1.65.1 libboost-math1.65.1 libboost-mpi1.65.1 libboost-numpy-dev
  libboost-numpy1.65-dev libboost-numpy1.65.1 libboost-program-options1.65.1
  libboost-python1.65.1

In [312]:
!git clone https://github.com/CAHLR/pyBKT.git

fatal: destination path 'pyBKT' already exists and is not an empty directory.


In [313]:
%cd /content/pyBKT

/content/pyBKT


In [314]:
!wget --no-check-certificate http://bitbucket.org/eigen/eigen/get/3.1.3.tar.gz
!tar -xzvf 3.1.3.tar.gz
!mv eigen-eigen-2249f9c22fe8 ./Eigen
!rm 3.1.3.tar.gz



URL transformed to HTTPS due to an HSTS policy
--2020-02-15 09:14:35--  https://bitbucket.org/eigen/eigen/get/3.1.3.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.2, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-02-15 09:14:36 ERROR 404: Not Found.

tar (child): 3.1.3.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
mv: cannot move 'eigen-eigen-2249f9c22fe8' to './Eigen/eigen-eigen-2249f9c22fe8': Directory not empty
rm: cannot remove '3.1.3.tar.gz': No such file or directory


In [315]:
!make

generating data...
fitting! each dot is a new EM initialization

	truth	learned
prior	0.1000	0.1011
learn1	0.3000	0.2818
forget1	0.0000	0.0000
guess1	0.1000	0.0995
slip1	0.0300	0.0287


In [0]:
#!python test/assistments_train.py #TODO: modify pre-procesd

In [0]:
#from google.colab import drive
#drive.mount('/content/drive/')
filepath='https://drive.google.com/uc?export=download&id=0B3f_gAH-MpBmUmNJQ3RycGpJM0k'

In [318]:
#Import all the dependencies
%cd /content
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.formula.api import ols
import scipy.stats as stats
import statsmodels.api as sm
import requests
from pyBKT.util import data_helper
#from pyBKT.generate import synthetic_data
from pyBKT.generate import random_model_uni
from pyBKT.fit import EM_fit
from sklearn.model_selection import train_test_split
from pyBKT.fit import predict_onestep
from sklearn.metrics import accuracy_score

/content


In [319]:
df = pd.read_csv(filepath,encoding='ISO-8859-1')
print('load')
df.head()

load


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.



,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,answer_type,sequence_id,student_class_id,position,type,base_sequence_id,skill_id,skill_name,teacher_id,school_id,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,algebra,5948,13241,126,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,32454,30799,NaN,26,0,NaN,1,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,algebra,5948,13241,126,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,4922,30799,NaN,55,0,NaN,2,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,42000,30799,NaN,88,0,NaN,1,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,0,3,4859,30059,NaN,41,0,NaN,2,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,algebra,5948,11816,22,MasterySection,5948,1.0,Box and Whisker,22763,73,3,4,124564,30060,NaN,65,0,0.0,3,3.0


In [320]:
print(df.dtypes)
print(df.shape)

order_id                  int64
assignment_id             int64
user_id                   int64
assistment_id             int64
problem_id                int64
original                  int64
correct                   int64
attempt_count             int64
ms_first_response         int64
tutor_mode               object
answer_type              object
sequence_id               int64
student_class_id          int64
position                  int64
type                     object
base_sequence_id          int64
skill_id                float64
skill_name               object
teacher_id                int64
school_id                 int64
hint_count                int64
hint_total                int64
overlap_time              int64
template_id               int64
answer_id               float64
answer_text              object
first_action              int64
bottom_hint             float64
opportunity               int64
opportunity_original    float64
dtype: object
(401756, 30)


In [321]:
df['skill_name'].value_counts()

Equation Solving Two or Fewer Steps         24253
Conversion of Fraction Decimals Percents    18742
Addition and Subtraction Integers           12741
Addition and Subtraction Fractions          11334
Percent Of                                   9497
                                            ...  
Distributive Property                          18
Finding Slope From Situation                    9
Recognize Quadratic Pattern                     8
Reading a Ruler or Scale                        5
Finding Slope from Ordered Pairs                5
Name: skill_name, Length: 110, dtype: int64

In [338]:
equ_sol = data_helper.assistments_data("Equation Solving Two or Fewer Steps")
perc_of = data_helper.assistments_data("Percent Of")
subtr = data_helper.assistments_data("Addition and Subtraction Integers")
fract = data_helper.assistments_data("Conversion of Fraction Decimals Percents")
prism = data_helper.assistments_data("Volume Rectangular Prism")

Equation Solving Two or Fewer Steps


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning:

Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Percent Of
Addition and Subtraction Integers
Conversion of Fraction Decimals Percents
Volume Rectangular Prism


In [323]:
np.unique(equ_sol['resources'])

array([1])

In [357]:
fitmodel = random_model_uni.random_model_uni(1,1)
fitmodel 

{'As': array([[[1.        , 0.        ],
         [0.16540696, 0.83459304]]]),
 'emissions': array([[[0.9975981 , 0.0024019 ],
         [0.02603143, 0.97396857]]]),
 'forgets': array([0.]),
 'guesses': array([0.05701317]),
 'learns': array([0.16540696]),
 'pi_0': array([[0.99891585],
        [0.00108415]]),
 'prior': 0.5566286234799914,
 'slips': array([0.04479041])}

In [363]:
 (fitmodel_equ, log_likelihoods_equ) = EM_fit.EM_fit(fitmodel, equ_sol)
 fitmodel_equ

{'As': array([[[0.96194317, 0.        ],
         [0.03805683, 1.        ]]]),
 'emissions': array([[[0.71383525, 0.28616475],
         [0.25115792, 0.74884208]]]),
 'forgets': array([0.]),
 'guesses': array([0.28616475]),
 'learns': array([0.03805683]),
 'pi_0': array([[0.36386579],
        [0.63613421]]),
 'prior': 0.6361342056575534,
 'slips': array([0.25115792])}

In [364]:
(fitmodel_p, log_likelihoods_p) = EM_fit.EM_fit(fitmodel,perc_of)
fitmodel_p

{'As': array([[[0.88263036, 0.        ],
         [0.11736964, 1.        ]]]),
 'emissions': array([[[0.92835778, 0.07164222],
         [0.28604927, 0.71395073]]]),
 'forgets': array([0.]),
 'guesses': array([0.07164222]),
 'learns': array([0.11736964]),
 'pi_0': array([[0.32826301],
        [0.67173699]]),
 'prior': 0.6717369914455578,
 'slips': array([0.28604927])}

In [365]:
(fitmodel_s, log_likelihoods_s) = EM_fit.EM_fit(fitmodel,subtr)
fitmodel_s

{'As': array([[[0.9908163, 0.       ],
         [0.0091837, 1.       ]]]),
 'emissions': array([[[0.63277772, 0.36722228],
         [0.19914567, 0.80085433]]]),
 'forgets': array([0.]),
 'guesses': array([0.36722228]),
 'learns': array([0.0091837]),
 'pi_0': array([[0.29528213],
        [0.70471787]]),
 'prior': 0.7047178665230069,
 'slips': array([0.19914567])}

In [366]:
 (fitmodel_f, log_likelihoods_f) = EM_fit.EM_fit(fitmodel,fract)
 fitmodel_f

{'As': array([[[0.98512679, 0.        ],
         [0.01487321, 1.        ]]]),
 'emissions': array([[[0.57634037, 0.42365963],
         [0.20192983, 0.79807017]]]),
 'forgets': array([0.]),
 'guesses': array([0.42365963]),
 'learns': array([0.01487321]),
 'pi_0': array([[0.39120669],
        [0.60879331]]),
 'prior': 0.6087933133028355,
 'slips': array([0.20192983])}

In [367]:
 (fitmodel_p, log_likelihoods_p) = EM_fit.EM_fit(fitmodel,prism)
 fitmodel_p

{'As': array([[[9.99998710e-01, 0.00000000e+00],
         [1.28970667e-06, 1.00000000e+00]]]),
 'emissions': array([[[0.6916403 , 0.3083597 ],
         [0.11840342, 0.88159658]]]),
 'forgets': array([0.]),
 'guesses': array([0.3083597]),
 'learns': array([1.28970667e-06]),
 'pi_0': array([[0.03666741],
        [0.96333259]]),
 'prior': 0.9633325916770059,
 'slips': array([0.11840342])}

In [0]:
df_2=df[df["original"] == 1]

In [328]:
train,test= train_test_split(df_2, test_size=0.5, random_state=42)
print(len(test), len(test))

164146 164146


In [0]:
def assistments_data_v2(dataset, skill_name):
  
  import pandas as pd
  import numpy as np
  import io
  import requests
  
  print(skill_name)
  #url = "https://drive.google.com/uc?export=download&id=0B3f_gAH-MpBmUmNJQ3RycGpJM0k"
  #s = requests.get(url).content
  # df = pd.read_csv(io.BytesIO(s))
  #df = pd.read_csv(io.StringIO(s.decode('latin')))
  # df = pd.read_csv(io.StringIO(s.decode('latin')), names = ['order_id', 'assignment_id', 'user_id', 'assistment_id', 'problem_id', 'original', 'correct', 'attempt_count', 'ms_first_response', 'tutor_mode', 'answer_type', 'sequence_id', 'student_class_id', 'position', 'type', 'base_sequence_id', 'skill_id', 'skill_name', 'teacher_id', 'school_id', 'hint_count', 'hint_total', 'overlap_time', 'template_id', 'answer_id', 'answer_text', 'first_action', 'bottom_hint', 'opportunity', 'opportunity_original'])
  # filter by the skill you want, make sure the question is an 'original'
  # skills = df["skill_name"]
  # for skill_name in skills:
  skill = dataset[(dataset["skill_name"]==skill_name)]
  # sort by the order in which the problems were answered
  dataset["order_id"] = [int(i) for i in dataset["order_id"]]
  dataset.sort_values("order_id", inplace=True)
  
  # example of how to get the unique users
  # uilist=skill['user_id'].unique()

  # convert from 0=incorrect,1=correct to 1=incorrect,2=correct
  skill.loc[:,"correct"]+=1
  
  # filter out garbage
  df3=skill[skill["correct"]!=3]
  data=df3["correct"].values
  
  # find out how many problems per user, form the start/length arrays
  steps=df3.groupby("user_id")["problem_id"].count().values
  lengths=np.copy(steps)
  # print("steps", steps)
  steps[0]=0
  steps[1]=1
  for i in range(2,steps.size):
    steps[i]=steps[i-1]+lengths[i-2]
  
  starts=np.delete(steps,0)

  resources=[1]*data.size
  resource=np.asarray(resources)
  
  stateseqs=np.copy(resource)
  lengths=np.resize(lengths,lengths.size-1)
  Data={}
  Data["stateseqs"]=np.asarray([stateseqs],dtype='int32')
  Data["data"]=np.asarray([data],dtype='int32')
  Data["starts"]=np.asarray(starts)
  Data["lengths"]=np.asarray(lengths)
  Data["resources"]=resource
  
  return (Data)

In [330]:
equ_train = assistments_data_v2(train,'Equation Solving Two or Fewer Steps')
equ_test = assistments_data_v2(test,'Equation Solving Two or Fewer Steps')
print(len(equ_train['data'][0]),"\n",len(equ_train['data'][0]))

Equation Solving Two or Fewer Steps
Equation Solving Two or Fewer Steps


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

8760 
 8760


In [331]:
(fitmodel_train, log_likelihoods_train) = EM_fit.EM_fit(fitmodel, equ_train)
print(fitmodel_train,log_likelihoods_train)

{'prior': 0.8635897209553083, 'learns': array([1.57312859e-05]), 'forgets': array([0.]), 'guesses': array([0.62570236]), 'slips': array([0.351489]), 'As': array([[[9.99984269e-01, 0.00000000e+00],
        [1.57312859e-05, 1.00000000e+00]]]), 'emissions': array([[[0.37429764, 0.62570236],
        [0.351489  , 0.648511  ]]]), 'pi_0': array([[0.13641028],
       [0.86358972]])} [[-6912.10627212]
 [-5697.84388314]
 [-5684.75041603]
 [-5682.01414155]
 [-5680.91270357]
 [-5680.3663513 ]
 [-5680.05618041]
 [-5679.86271543]
 [-5679.73342243]
 [-5679.64234808]
 [-5679.57548745]
 [-5679.52474196]
 [-5679.48516057]
 [-5679.45357497]
 [-5679.42787875]
 [-5679.40662542]
 [-5679.38879369]
 [-5679.3736449 ]
 [-5679.36063338]
 [-5679.34934851]
 [-5679.33947611]
 [-5679.33077221]
 [-5679.32304483]
 [-5679.31614105]
 [-5679.30993776]
 [-5679.30433482]
 [-5679.29925006]
 [-5679.29461545]
 [-5679.29037425]
 [-5679.2864788 ]
 [-5679.28288879]
 [-5679.27956991]
 [-5679.27649281]
 [-5679.27363222]
 [-5679.27

In [332]:
pred = predict_onestep.run(fitmodel_train,equ_test)
pred

(array([0.64673408, 0.63232984, 0.64662323, ..., 0.07959498, 0.54610737,
        0.08406475]),
 array([[1.36410279e-001, 8.63589721e-001, 1.43985322e-001, ...,
         1.27209018e-001, 8.72790982e-001, 1.34352619e-001],
        [8.65647381e-001, 1.41832169e-001, 8.58167831e-001, ...,
         6.95253303e-310, 6.91590974e-310, 5.09257572e-317]]))

In [333]:
pred_1 = pd.core.series.Series(pred[0]).apply(lambda x : 1 if ( x >= 0.5) else 0)+1
pred_1

0       2
1       2
2       2
3       2
4       2
       ..
8546    1
8547    2
8548    1
8549    2
8550    1
Length: 8551, dtype: int64

In [334]:
accuracy_score(pred_1,equ_test['data'][0])

0.6457724242778622